In [1]:
from BaseHTTPServer import BaseHTTPRequestHandler
import urlparse, json
import sys
import simplejson
import yaml
class GetHandler(BaseHTTPRequestHandler):
    
    def parse_avail(self,jsonfile):    
        js_list = []

        for line in jsonfile:
            ele = line.split()
            if ele[0] == '[' or ele[0] == ']':
                continue
            elif ele[0] == '{':
                js_line = '{'
            elif ele[0] == '},':
                js_line += '}'
                j_dict = json.loads(js_line)
                js_list.append(j_dict)
            else:
                js_line += line
        jsonfile.close()
        return js_list

    def do_GET(self):
        message = ''
        parsed_path = urlparse.urlparse(self.path)
        print parsed_path.query
        if parsed_path.query == 'clothing_list':
            try:
                shopfile = open('./clothing_list.txt','r')
            except:
                print('Error: An error occured trying to read the file.')
                sys.exit()
            js_list = self.parse_avail(shopfile)
            message = js_list
            
        if parsed_path.query == 'order_history':
            try:
                orderfile = open('./test.txt','r')
            except:
                print('Error: An error occured trying to read the file.')
                sys.exit()
            js_list = simplejson.load(orderfile)     
            orderfile.close()
            message = js_list
            
        self.send_response(200)
        self.end_headers()
        self.wfile.write(message) # write a response back to the client
        return

    def do_POST(self):
        content_len = int(self.headers.getheader('content-length')) # length of request body
        post_body = self.rfile.read(content_len) # format [{},{},...,{}]
        self.send_response(200)
        self.end_headers()
        data = yaml.load(post_body) # get a list of dictionaries
        with open("test.txt", "w") as outfile:
            simplejson.dump(data, outfile)    
        self.wfile.write('Order confirmed\n')
        return

if __name__ == '__main__':
    from BaseHTTPServer import HTTPServer
    server = HTTPServer(('localhost', 8080), GetHandler)
    print 'Starting server at http://localhost:8080'
    server.serve_forever()

Starting server at http://localhost:8080



127.0.0.1 - - [20/Jul/2017 16:37:41] "GET / HTTP/1.1" 200 -


127.0.0.1 - - [20/Jul/2017 16:37:41] "GET /favicon.ico HTTP/1.1" 200 -


127.0.0.1 - - [20/Jul/2017 16:37:51] "GET /? HTTP/1.1" 200 -


127.0.0.1 - - [20/Jul/2017 16:37:52] "GET /favicon.ico HTTP/1.1" 200 -


...

127.0.0.1 - - [20/Jul/2017 16:37:57] "GET /?clothing_list HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2017 16:37:57] "GET /favicon.ico HTTP/1.1" 200 -


order_history


127.0.0.1 - - [20/Jul/2017 16:38:04] "GET /?order_history HTTP/1.1" 200 -


127.0.0.1 - - [20/Jul/2017 16:38:05] "GET /favicon.ico HTTP/1.1" 200 -


KeyboardInterrupt: 